In [16]:
#! pip install langdetect rapidfuzz cleantext

In [2]:
import os #interaction with the operating system
import pandas as pd #for data manipulation, use pd when referring to panda
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import requests #library for HTTP requests, used to retrieve data from web APIs
import nltk #NLTK=Natural Language ToolKit, for working with text data, used for text processing such as tokenizatino, stemming, tagging, ...
from collections import namedtuple #import tuple library from the collections module
from nltk.corpus import brown #import brown from the nltk module, collection of english text
import time #work with time-related functions, e.g., execution time
import re #regular expression library, support for searching, matching and manipulating strings
import urllib.parse #parsing URLs, splitting them into components
import html #for decoding HTML entities to convert them back to normal characters
import re #for cleaning and removing

from langdetect import detect, DetectorFactory
from tqdm import tqdm
from rapidfuzz import fuzz
from cleantext import clean

2025-03-13 19:14:46.345258: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-13 19:14:46.360225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741889686.377723  605164 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741889686.383072  605164 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 19:14:46.401009: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
# Ensure NLTK resources are downloaded
nltk.download('brown') #Brown Corpus of the nltk toolkit is downloaded, this is A dataset of text that is commonly used in NLP tasks

# Define helper functions
def to_abstract(paper): 
    if 'abstract_inverted_index' in paper and paper['abstract_inverted_index'] is not None: #Check to see if the value of the key is None
        #return ' '.join(paper['abstract_inverted_index'].keys())
        word_index = []
        for k,v in paper['abstract_inverted_index'].items():
            for index in v:
                word_index.append([k,index])
    
        word_index = sorted(word_index,key = lambda x : x[1])
    
        word_list = []
        for i in range(len(word_index)):
            word_list.append(word_index[i][0])
    
        separator = ' '
        reconstructed_text = separator.join(word_list)
    
        return reconstructed_text
    else:
        return ''
        

def is_english(text, brown_corpus, threshold=0.5): #threshold determines the minimum percentage of English words
    """Determine if the given text is in English."""
    if not text:
        return 'EMPTY', 'NA'

    try:
        tokens = set(re.findall(r"\w+", text.lower()))
        if not tokens:
            return 'EMPTY', 'NA'

        lang_ratio = len(tokens & brown_corpus.value) / len(tokens)
        return 'PASS', 'en' if lang_ratio > threshold else 'non-en'
    except Exception as e:
        return 'FAIL', str(e)

# Clean text and decode HTML entities
def clean_text(text):
    """Clean text by decoding HTML entities and fixing encoding issues."""
    text = html.unescape(text)
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)  # Removes anything in angle brackets, e.g., <div>, <h>
    # Remove specific unwanted prefixes, e.g., "e17029 Background:"
    text = re.sub(r'\b[e0-9]+\s+Background:\s*', '', text)  # Remove patterns like "e17029 Background:"

    # Encode in 'latin1' (ISO-8859-1) and decode to 'utf-8' to fix special characters
    try:
      text = text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
      pass

    return text

def format_ids(items, base_url):
    """Format IDs from a list of items."""
    return '|'.join([item['id'].replace(base_url, '') for item in items if 'id' in item])

def format_authors(authorships):
    """Extract author names from the authorship field."""
    # Clean author names with clean_text()
    return ', '.join([clean_text(authorship['author']['display_name']) for authorship in authorships if 'author' in authorship])

def format_citations(referenced_works):
    """Extract and format citations (referenced works) from the paper."""
    return '|'.join([ref_work.replace('https://openalex.org/W', '') for ref_work in referenced_works])

def fetch_articles(base_url, headers):
    """Fetch articles from the OpenAlex API."""
    cursor = '*'
    papers_list = []
    i = 0

    while cursor:
        full_url = f"{base_url}&per_page=100&cursor={cursor}"
        print(f"Fetching results ({i})...")
        i = i+100

        response = requests.get(full_url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            if 'results' in data and data['results']:
                for paper in data['results']:
                    # Check if paper is None
                    if paper is None:
                        print("Skipping None paper") # Print a message to identify skipped papers
                        continue # Skip to the next paper

                    abstract = to_abstract(paper)
                    title = paper.get('title', '')

                    # Check if title is None or empty before calling replace
                    if title:
                        title = title.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
                    pub_date = paper.get('publication_date', '')
                    pub_year = paper.get('publication_year', '')

                    
                    
                    # Ensure both title and abstract are strings before concatenation
                    if title is None:
                        title = ''
                    if abstract is None:
                        abstract = ''

                    # Clean the title and abstract using clean_text()
                    title = clean_text(title)
                    abstract = clean_text(abstract)

                    status, lang = is_english(title + ' ' + abstract, bc_brown)
                    
                    if lang == 'en' and len(title)>0 and len(abstract)>0:
                    
                        authors = format_authors(paper.get('authorships', []))
                        citations = format_citations(paper.get('referenced_works', []))
                        papers_list.append({
                            'PaperId': paper['id'].replace('https://openalex.org/W', ''),
                            'PaperTitle': title,
                            'Citations': citations,
                            'coFoS': format_ids(paper.get('concepts', []), 'https://openalex.org/C'),
                            'Authors': authors,
                            'Abstract': abstract,
                            'Lang': 'en',
                            'PubYear': pub_year,
                            'PubDate': pub_date
                        })
                cursor = data['meta'].get('next_cursor', None)
            else:
                cursor = None

            time.sleep(1)  # Pause to avoid rate limiting
        else:
            print(f"Request failed with status code {response.status_code}")
            break

    return papers_list


[nltk_data] Downloading package brown to /home/janna/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [5]:
# Initialize parameters
Brown = namedtuple("Brown", field_names=['value'])
bc_brown = Brown(value=set(word.lower() for word in brown.words()))
headers = {'email': 'janna.hastings@uzh.ch'}

In [14]:
# Define search parameters
search_term = '("mental" OR "psychological" OR "behavioural" OR "psychology" OR "psychiatry" OR "neurological" OR "mind" OR "brain" OR "behaviour" OR "psychiatric") \
              AND ("anxiety" OR "depression" OR "psychosis") AND ("treatment" OR "therapy" OR "therapeutic" OR "mechanism" OR "intervention" OR "early" OR "diagnosis" OR "diagnostic" OR "translation")'

publication_year_range = '2015-2025' #10 years
encoded_search_term = urllib.parse.quote(search_term)

base_url = f'https://api.openalex.org/works?filter=has_abstract:true,title_and_abstract.search:{encoded_search_term},publication_year:{publication_year_range}&sort=publication_year:desc'

# Create the output directory if it does not exist
output_dir = './output'
os.makedirs(output_dir, exist_ok=True)

In [15]:
# Fetch and process articles
papers = fetch_articles(base_url, headers)


Fetching results (0)...
Fetching results (100)...
Fetching results (200)...
Fetching results (300)...
Fetching results (400)...
Fetching results (500)...
Fetching results (600)...
Fetching results (700)...
Fetching results (800)...
Fetching results (900)...
Fetching results (1000)...
Fetching results (1100)...
Fetching results (1200)...
Fetching results (1300)...
Fetching results (1400)...
Fetching results (1500)...
Fetching results (1600)...
Fetching results (1700)...
Fetching results (1800)...
Fetching results (1900)...
Fetching results (2000)...
Fetching results (2100)...
Fetching results (2200)...
Fetching results (2300)...
Fetching results (2400)...
Fetching results (2500)...
Fetching results (2600)...
Fetching results (2700)...
Fetching results (2800)...
Fetching results (2900)...
Fetching results (3000)...
Fetching results (3100)...
Fetching results (3200)...
Fetching results (3300)...
Fetching results (3400)...
Fetching results (3500)...
Fetching results (3600)...
Fetching resu

In [17]:
# Save raw search results 
output_file_path = os.path.join(output_dir, 'searchresults.csv')
df = pd.DataFrame(papers)
df.to_csv(output_file_path, index=False)
print(f'Done. Fetched {len(df)} papers. File saved to {output_file_path}')

Done. Fetched 182747 papers. File saved to ./output/searchresults.csv


In [18]:
# Clean the search results data
print("Before cleaning step 1: ",len(df))

#df = df[df['Authors'].notna() & (df['Authors'] != '')].reset_index(drop=True)
#df = df[ ~df['PaperTitle'].str.lower().str.startswith((
#        'supplem', 'data from', 'data to', 'datasheet', 'dataset', 
#        'supporting figure', 'supp fig', 'supp table', 'supp.', 'suppl.', 'suppl data', 'suppl fig',
#        'data supplement', 'fig.', 'figure'
#    ))].reset_index(drop=True)
#df = df[ df['Abstract'].str.len() < 32000 ].reset_index(drop=True)  
#df = df.drop_duplicates(subset=['PaperTitle', 'Authors', 'PubDate'])


# Define cleaning steps and initialize tqdm progress bar
cleaning_steps = [
    "Removing missing/empty PaperId",
    "Removing duplicate PaperId entries",
    "Removing duplicate entries with identical PaperTitle & Authors",
    "Removing missing/empty PaperTitle",
    "Removing missing/empty Abstract",
    "Removing missing/empty PubDate",
    "Validating PubDate format (YYYY-MM-DD)",
    "Removing missing/empty Authors"
]
progress_bar = tqdm(total=len(cleaning_steps), desc="Cleaning Data", unit="step")

# Count initial number of rows
initial_article_count = len(df)

# Removing missing/empty PaperId
missing_paperid_count = df['PaperId'].isna().sum()
empty_paperid_count = (df['PaperId'].astype(str).str.strip() == "").sum()
df = df.dropna(subset=['PaperId'])
df = df[df['PaperId'].astype(str).str.strip() != ""]
progress_bar.update(1)

# Removing missing/empty PaperTitle
missing_title_count = df['PaperTitle'].isna().sum()
empty_title_count = (df['PaperTitle'].str.strip() == "").sum()
df = df.dropna(subset=['PaperTitle'])
df = df[df['PaperTitle'].str.strip() != ""]
progress_bar.update(1)

# Removing missing/empty Abstract
missing_abstract_count = df['Abstract'].isna().sum()
empty_abstract_count = (df['Abstract'].str.strip() == "").sum()
df = df.dropna(subset=['Abstract'])
df = df[df['Abstract'].str.strip() != ""]
progress_bar.update(1)

# Removing missing/empty Authors
missing_authors_count = df['Authors'].isna().sum()
empty_authors_count = (df['Authors'].astype(str).str.strip() == "").sum()
df = df.dropna(subset=['Authors'])
df = df[df['Authors'].astype(str).str.strip() != ""]
progress_bar.update(1)

# Removing missing/empty PubDate
missing_pubdate_count = df['PubDate'].isna().sum()
empty_pubdate_count = (df['PubDate'].astype(str).str.strip() == "").sum()
df = df.dropna(subset=['PubDate'])
df = df[df['PubDate'].astype(str).str.strip() != ""]
progress_bar.update(1)

# Validate PubDate format (YYYY-MM-DD)
def is_valid_date(date_str):
    return bool(re.match(r'^\d{4}-\d{2}-\d{2}$', str(date_str)))
invalid_pubdate_count = df[~df['PubDate'].astype(str).apply(is_valid_date)].shape[0]
df = df[df['PubDate'].astype(str).apply(is_valid_date)]
progress_bar.update(1)

# Removing duplicate PaperId entries
duplicate_paperid_count = df.duplicated(subset=['PaperId']).sum()
df = df.drop_duplicates(subset=['PaperId'], keep='first')
progress_bar.update(1)

# Removing duplicate entries with identical PaperTitle & Authors
df["_temp_PaperTitle"] = df["PaperTitle"].str.lower()
df["_temp_Authors"] = df["Authors"].str.lower()
duplicate_entries_count = df.duplicated(subset=["_temp_PaperTitle", "_temp_Authors"]).sum()
df = df.drop_duplicates(subset=["_temp_PaperTitle", "_temp_Authors"], keep="first")
df.drop(columns=["_temp_PaperTitle", "_temp_Authors"], inplace=True)
progress_bar.update(1)  


# Close the progress bar
progress_bar.close()

# Count final number of rows after all removals
article_count_after_duplicates_and_missings = len(df)
total_removed = initial_article_count - article_count_after_duplicates_and_missings

# Sum the duplicate removals
total_duplicates_removed = duplicate_paperid_count + duplicate_entries_count

# Sum the missing removals
missing_values = (
    missing_paperid_count + empty_paperid_count +
    missing_title_count + empty_title_count +
    missing_abstract_count + empty_abstract_count +
    missing_pubdate_count + empty_pubdate_count + invalid_pubdate_count +
    missing_authors_count + empty_authors_count
)

# Collect removal statistics
removal_log = [
    f"Initial number of articles: {initial_article_count:,}",
    f"\n-Removed due to missing PaperId (NaN): {missing_paperid_count:,}",
    f"-Removed due to empty PaperId (''): {empty_paperid_count:,}", 
    f"-Removed due to missing PaperTitle (NaN): {missing_title_count:,}",
    f"-Removed due to empty PaperTitle (''): {empty_title_count:,}",
    f"-Removed due to missing Abstract (NaN): {missing_abstract_count:,}",
    f"-Removed due to empty Abstract (''): {empty_abstract_count:,}",
    f"-Removed due to missing PubDate (NaN): {missing_pubdate_count:,}",
    f"-Removed due to empty PubDate (''): {empty_pubdate_count:,}",
    f"-Removed due to invalid PubDate format (not YYYY-MM-DD): {invalid_pubdate_count:,}",
    f"-Removed due to missing Authors (NaN): {missing_authors_count:,}",
    f"-Removed due to empty Authors (''): {empty_authors_count:,}",
    f"-Removed due to duplicate PaperId entries: {duplicate_paperid_count:,}",
    f"-Removed due to duplicate entries with identical PaperTitle & Authors: {duplicate_entries_count:,}", 
    
    f"\nTotal articles with missing data removed: {missing_values:,}", 
    f"Total duplicate entries removed: {total_duplicates_removed:,}", 
    f"Total articles removed: {total_removed:,}",
    
    f"\nFinal number of articles after cleaning steps: {article_count_after_duplicates_and_missings:,}"
]

# Print removal details
print("\n".join(removal_log))

output_file_path = os.path.join(output_dir, 'searchresults-cleaned.csv')
df.to_csv(output_file_path, index=False)

print(f'File saved to {output_file_path}')

Before cleaning step 1:  182747


Cleaning Data: 100%|████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.04step/s]


Initial number of articles: 182,747

-Removed due to missing PaperId (NaN): 0
-Removed due to empty PaperId (''): 0
-Removed due to missing PaperTitle (NaN): 0
-Removed due to empty PaperTitle (''): 1
-Removed due to missing Abstract (NaN): 0
-Removed due to empty Abstract (''): 0
-Removed due to missing PubDate (NaN): 0
-Removed due to empty PubDate (''): 0
-Removed due to invalid PubDate format (not YYYY-MM-DD): 0
-Removed due to missing Authors (NaN): 0
-Removed due to empty Authors (''): 2,279
-Removed due to duplicate PaperId entries: 0
-Removed due to duplicate entries with identical PaperTitle & Authors: 3,750

Total articles with missing data removed: 2,280
Total duplicate entries removed: 3,750
Total articles removed: 6,030

Final number of articles after cleaning steps: 176,717
File saved to ./output/searchresults-cleaned.csv


In [19]:
# Cleaning step 2 

# Progress bar
tqdm.pandas()

# Ensure consistent language detection (fixes randomness)
DetectorFactory.seed = 0  

# Start tracking runtime
start_time = time.time()

# Function to detect language separately for PaperTitle and Abstract with retry logic
def detect_language(row):
    try:
        title = str(row['PaperTitle']).strip().lower() if pd.notna(row['PaperTitle']) else ""
        abstract = str(row['Abstract']).strip() if pd.notna(row['Abstract']) else ""

        # Apply language detection with retry mechanism
        def safe_detect(text, max_retries=3):
            for _ in range(max_retries):
                try:
                    return detect(text) if len(text) > 10 else "unknown"  # Ignore very short text
                except:
                    pass
            return "error"  # If all retries fail, mark as error

        title_lang = safe_detect(title)
        abstract_lang = safe_detect(abstract)

        # Keep if AT LEAST ONE is English
        return "en" if title_lang == "en" or abstract_lang == "en" else "non-en"

    except Exception:
        return "error"  # Suppresses error messages

# Apply language detection with progress bar on the full dataset
df["detected_language"] = df.progress_apply(detect_language, axis=1)

# Separate English and non-English articles (Keep if AT LEAST ONE column is English)
df_filtered_cleaned = df[df["detected_language"] == "en"].drop(columns=["detected_language"])
df_dropped = df[df["detected_language"] != "en"]

# Store lengths in variables
total_rows_before = len(df)
dropped_rows = len(df_dropped)
total_rows_after = len(df_filtered_cleaned)

# Stop tracking runtime
end_time = time.time()
runtime_seconds = end_time - start_time
runtime_minutes = runtime_seconds / 60  # Convert seconds to minutes

# Save results
df_dropped.to_csv(os.path.join(output_dir, "searchresults_non_english.csv"), index=False)
df_filtered_cleaned.to_csv(os.path.join(output_dir, "searchresults-cleaned.csv"), index=False)

# Final print statement
print(f"Language filtering complete. Non-English articles removed: {dropped_rows:,}.")
print(f"Total time: {runtime_seconds:.2f} seconds ({runtime_minutes:.2f} minutes).")

100%|██████████████████████████████████████████████████████████████████████████████████████| 176717/176717 [21:14<00:00, 138.69it/s]


Language filtering complete. Non-English articles removed: 583.
Total time: 1274.26 seconds (21.24 minutes).


In [20]:
# Start tracking runtime
tqdm.pandas()
start_time = time.time()

# Keywords to match at the start of the PaperTitle column (sorted, formatted)
all_keywords = sorted([
    "Acknowledgments", "ADOBE PDF", "Addendum", "Advertisement", "Additional figure", "Additional Material", 
    "Additional Table", "All Supplemental Data", "All Supplemental Figures", 
    "All Supplemental Figures, Tables and Legends", "All Supplementary Data", 
    "All Supplementary Figures", "All Supplementary Figures and Tables", 
    "All Supplementary Tables, Figures and Methods", "Analysis Methods Supplement", 
    "Appendix", "Article figures", "Author Correction", "Auxiliary Supplementary File", 
    "Caption for Suppl Fig.", "Captions of supplementary", "Combined supplemental figures", 
    "Contents Vol", "Contents:",
    "Corrigendum", "Correction", "Correction to", "Correction:", "Corrections to", 
    "Data Augmentation", "Data associated with", "Data and code", "Data and analysis scripts", "Data File", 
    "Data for", "Data from", "Data not shown", "Data and metadata",
    "Data on", "Data S", "Data Spreadsheet", "Data Supplement", "Data.zip", 
    "Dataset", "Dataset and metadata", "Dataset for", "Dataset related to", 
    "Description of Supplemental Figures", "Description of supplementary", 
    "Download Supplementary", "eFigure", "Erratum", "Extended Data", "Expression of Concern",
    "Fig", "Fig.", "Fig Supp", "FigS", "Figure", "Figure S", "Figures", "Instructions for Authors",
    "File", "Funding statement", "http://", "https://", 
    "Legend for", "Legend from", "Legend of", "Legend to", 
    "Legends for", "Legends from", "Legends of", "Legends to", 
    "Legends Supplemental Figures", "Link to Supplementary", 
    "List of figures", "List of tables", "List of plates", "Manuscript Figures", 
    "Mat & Met", "Material and Methods", "Materials and Methods", 
    "Merged Supplementary", "Metadata and data", "Metadata record for", 
    "Metadata supporting", "Methods References", "Methods and Materials from", 
    "Methods and Supplementary", "Methods and Tables", "Methods file", 
    "Methods from", "Methods, Figures", "Methods, Table", "MET supplementary methods", 
    "methods_", "Movie", "Multimedia", "Methodology and Method", "Online supplement", 
    "Online Supplementary Materials", "Online Supplementary Tables", 
    "Online-only supplementary", "Online-Only Tables", 
    "Original Western Blot Data", "Revised Supplementary", "S Figure", "S Table", 
    "S-Figure", "S1 Legend", "S1 from", "S1.", "S1:", "S2 from", "S2.", "S2:", 
    "S3 from", "S3.", "S3:", "S4 from", "S4.", "S4:", "S5 from", "S5.", "S5:", 
    "S6 from", "S6.", "S6:", "S7 from", "S7.", "S7:", "S8 from", "S8.", "S8:", 
    "S9 from", "S9.", "S9:", "SI Figure", "SI Methods", "SI Table", 
    "SI materials", "SI tables and figures", "SFigure", "SF-", "SF1", 
    "Sl Figure", "Supplement", "Supplement Material", "Supplement Table", 
    "Supplemental", "Supplemental Data", "Supplemental Figure", 
    "Supplemental Figures", "Supplemental Table", "Supplementary", 
    "Supplementary Data", "Supplementary Figure", "Supplementary Figures", 
    "Supplementary Legends", "Supplementary Methods", "Supplementary Movie", 
    "Supplementary Table", "Supplementary Tables", "Supplementary Text", 
    "Supplementary Video", "SupplementaryFigures", "Supplementary_Figure", 
    "Supplementray Figures", "Supplementray Tables", "Supplymentary Figure", 
    "Supplymentary Figures", "Supplymentary Table", "Supporting Data", 
    "Supporting Document", "Supporting Figure", "Supporting Figures", 
    "Supporting File", "Supporting Information", "Supporting Legend", 
    "Supporting Legends", "Supporting Materials", "Supporting Methods", 
    "Supporting Table", "Supporting Tables", "Supporting Text", 
    "Supporting Video", "Suppl", "Suppl Figure", "Suppl Figures", "Suppl File", 
    "Suppl Info", "Suppl Information", "Suppl Legends", "Suppl List of Videos, Tables and Methods", 
    "Suppl Materials and Methods", "Suppl Methods", "Suppl Movie", "Suppl Table", 
    "Suppl Video", "Supplimentary Figure", "Supplimentary Methods", 
    "Supplimentary Tables", "Supplimentary_Figure", "Suplementary", "Supllementary Figure", 
    "Suplplementary Figure Legend", "Suplpementary Table", "Supplrmentary Figures", 
    "Supplrmentary Tables", "Suppltable", "Suppltext", "Supplvideo", 
    "Suppl_Figure", "Suppl-Figure", "Suupplementary Tables", "Table", "Table of Content",
    "Validation Figures", "Video", "Western Blots from", "Whole Slide Image"
])

# Keywords to match anywhere in the PaperTitle column (sorted, formatted)
middle_keywords = sorted([
    "Combined Supplementary Materials", "Corrigendum", "The Case Files", "Additional file",
    "Figure S", "Figures S", "Manuscript Figures", "SI tables and figures", 
    "Supplemental Data", "Supplemental figures", "Supplemental from", 
    "Supplemental table", "Supplemental figure", "Supplementary Figure", 
    "Supplementary Figures", "Supplementary Material", "Supplementary Methods", 
    "Supplementary Table", "Supplementary Tables", "Supplementary data", 
    "Supplementary document", "Supplementary information", "SupplementaryFigures", 
    "Supplementary materials and methods", 
    "Translation for the Article", "Video Dataset", "Operative Video"
])


# Compile regex patterns for faster execution
start_pattern = re.compile(r"^(?:" + "|".join(map(re.escape, all_keywords)) + r")", re.IGNORECASE)
middle_pattern = re.compile(r"\b(?:{})\b".format("|".join(map(re.escape, middle_keywords))), re.IGNORECASE)
supplementary_pattern = re.compile(r"[\w\-_]*supple\w*", re.IGNORECASE)

# Count initial rows
rows_before_artifacts = len(df_filtered_cleaned)

# Apply filtering with a progress bar
tqdm.pandas(desc="Filtering Data")

df_filtered_cleaned = df_filtered_cleaned[
    ~df_filtered_cleaned["PaperTitle"].progress_apply(lambda title: bool(start_pattern.match(str(title)))) &
    ~df_filtered_cleaned["PaperTitle"].progress_apply(lambda title: bool(middle_pattern.search(str(title)))) &
    ~df_filtered_cleaned["PaperTitle"].progress_apply(lambda title: "_" in str(title)) &
    ~df_filtered_cleaned["PaperTitle"].progress_apply(lambda title: bool(supplementary_pattern.search(str(title))))
]

# Count remaining and removed rows
final_row_count = len(df_filtered_cleaned)
artifacts_removed = rows_before_artifacts - final_row_count

# Display results
print(f"\nInitial number of rows before artifact removal: {rows_before_artifacts:,}")
print(f"Number of rows identified as artifacts and removed: {artifacts_removed:,}")
print(f"Final number of rows in the cleaned dataset: {final_row_count:,}")

# Save the cleaned dataset
df_filtered_cleaned.to_csv(os.path.join(output_dir, "searchresults-cleaned.csv"), index=False)

# End tracking runtime
end_time = time.time()
print(f"Processing completed in {end_time - start_time:.2f} seconds.")

Filtering Data: 100%|████████████████████████████████████████████████████████████████████| 176134/176134 [00:02<00:00, 68763.17it/s]



Initial number of rows before artifact removal: 176,134
Number of rows identified as artifacts and removed: 791
Final number of rows in the cleaned dataset: 175,343
Processing completed in 18.43 seconds.


In [21]:
# Initialize removal counts
nonsense_title_counts = 0
title_length_counts = 0
abstract_length_counts = 0
withdraw_counts = 0
non_research_counts = 0

# Define cleaning steps for progress bar
new_cleaning_steps = [
    "Removing numeric/nonsense titles",
    "Removing placeholder titles",
    "Filtering by title length",
    "Filtering by abstract length",
    "Removing withdrawn/withdrawal articles",
    "Removing non-research articles (Editorials, News, etc.)"
]

progress_bar = tqdm(total=len(new_cleaning_steps), desc="Additional Cleaning", unit="step")

# ========== Removing numeric/nonsense titles ==========
pattern_numeric = re.compile(r'^[^A-Za-z]+$')  # Matches titles with no letters (only numbers/symbols)
mask_numeric = df_filtered_cleaned["PaperTitle"].apply(lambda title: bool(pattern_numeric.match(str(title))))
nonsense_title_counts += mask_numeric.sum()
df_filtered_cleaned = df_filtered_cleaned[~mask_numeric]
progress_bar.update(1)
print(f"Identified numeric/nonsense titles: {mask_numeric.sum():,}")

# ========== Removing placeholder titles ==========
placeholders = {
    "Title Not Available", "No Title", "Untitled", "N/A",
    "[No title available]", "No title available", "No title available - PubMed",
    "Error in Text", "Errors in Box", "Error in Author Name", "Error in Author Names",
    "Error in Author Surnames", "Errors in article text", "Error in Figure Label and Caption",
    "Error in Presentation of Author", "Errors in Author Name", "Error in Author Name",
    "Error in Table Text","Error in Corresponding Authorship",
}

placeholder_pattern = re.compile(
    r"\b(?:{})\b".format("|".join(map(re.escape, placeholders))), re.IGNORECASE
)
mask_placeholders = df_filtered_cleaned["PaperTitle"].apply(lambda title: bool(placeholder_pattern.search(str(title))))
print(f"Identified placeholders: {mask_placeholders.sum():,}")
nonsense_title_counts += mask_placeholders.sum()
df_filtered_cleaned = df_filtered_cleaned[~mask_placeholders]
progress_bar.update(1)

# ========== Filtering by title length ==========
def count_real_words(text):
    words = re.findall(r'\b[a-zA-Z]{2,}\b', str(text))  # Only words with 2+ letters
    return len(words)

def valid_title_length(title, min_words=3, max_words=60):
    return min_words <= count_real_words(title) <= max_words

df_filtered_cleaned = df_filtered_cleaned.reset_index(drop=True)
mask_valid_title = df_filtered_cleaned["PaperTitle"].apply(valid_title_length)

title_length_counts = (~mask_valid_title).sum()
print(f"Invalid PaperTitle count (too short/long): {title_length_counts:,}")

# Remove invalid titles first
df_filtered_cleaned = df_filtered_cleaned[mask_valid_title]
progress_bar.update(1)

# ========== Filtering by abstract length ==========
def valid_abstract_length(abstract, min_words=80, max_words=1000):
    return min_words <= count_real_words(abstract) <= max_words

mask_valid_abstract = df_filtered_cleaned["Abstract"].apply(valid_abstract_length)
abstract_length_counts = (~mask_valid_abstract).sum()

print(f"Invalid Abstract count (too short/long): {abstract_length_counts:,}")

# Remove invalid abstracts (after title filtering)
df_filtered_cleaned = df_filtered_cleaned[mask_valid_abstract]
progress_bar.update(1)

# ========== Removing withdrawn/retracted papers ==========
withdrawn_retraction_pattern = re.compile(
    r"(?i)^(?:\[?)?(Withdrawn|Withdrawal|Retracted|Retraction|Errata|Erratum|Revoked|Removed|Correction Notice|Corrigendum)\b"
)
mask_withdrawn = df_filtered_cleaned["PaperTitle"].apply(lambda title: bool(withdrawn_retraction_pattern.search(str(title))))
withdraw_counts = mask_withdrawn.sum()
df_filtered_cleaned = df_filtered_cleaned[~mask_withdrawn]
progress_bar.update(1)
print(f"Identified withdrawn: {withdraw_counts:,}")


# ========== Removing non-research articles ==========
non_research_keywords = [
    "A reply to Letter to the editor", "About the Author", "An Update from the Editor-in-Chief",
    "Announcement:", "Announcements", "Appeal", "ASO Author Reflection", "AUTHOR COPY ONLY", 
    "Associate Editor", "Author comment:", "Author profile", "Author Reflections:", "Author reply", 
    "Author response", "Author response to", "Author view", "Author's reply", "Author's respond", 
    "Editor's Reflection", "Editors Reflection", "Editors' Reflection", "Editor' Reflection",
    "Author's response", "Author's view", "Authors reply", "Authors respond", "Authors response", 
    "Authors view", "Authors' Reply", "Authors' response", "Authors's response", "Book Review", "EditorinChief",
    "Comment", "Comment on", "Commentary", "Conference Summary", "Conversation with the Editor", 
    "Correspondence", "Editorial", "Editor Profile", "Editor response for", "Editor change", "New Editor",
    "Editor-in-Chief", "Editorinchief's introduction", "Editor's evaluation", "Editor's Introduction", 
    "Editor's Message", "Editor's note", "Editors Introduction", "Editors Message", "Editors Note on", 
    "Editors Spotlight", "Editors change", "Editors note", "Editors' Introduction", "Editors' Message", 
    "Editors' note", "Foreword", "From the Editor", "From the Editor-in-Chief", "From the Editors...", 
    "From the Editors Desk...", "From the Editor's desk", "From the editors desk", "From the Guest Editor", "Guest editor",
    "Guest editorial", "Introducing our new Editors", "Issue Editor Foreword", "Issue Information Editorial Board", 
    "Letter to editor", "Letter to the Editor", "Letter:", "Letter Of The Editor", "Letters of the editor", "Letter-to-the-editor", 
    "Letters to editor", "Letters to the editor", "Meet the First Author", "Meet the author", "Meet the editor", 
    "Message from Editor", "Message from the Editor", "News", "Opinion", "our author", "our editor", 
    "Reply:", "In reply:", "Reply to:", "In reply to:",
    "Perspective", "Preface", "Proceedings", "Publisher'?s? Note", "Reply by Author", "Reply by the author", 
    "Reply to Editorial Comment", "Reply to Letter to", "Reply to correspondence", "Reply to the Letter to", 
    "Response to Editorial", "Response to a letter", "Response to letter", "senior editor", "Special Section", 
    "The Author Reply", "The Authors Reply", "To the Editor", "Transitioning Between Editor", "Viewpoint", 
    "Welcome to our", "Executive Editor","Assistant Editor",
]


# Convert keywords into a single regex pattern (match anywhere in the title)
escaped_keywords = [re.escape(keyword) for keyword in non_research_keywords]
non_research_pattern = "|".join(escaped_keywords)  # No caret (^) to match anywhere

df_filtered_cleaned["PaperTitle"] = df_filtered_cleaned["PaperTitle"].fillna("").str.strip().str.replace(r"\s+", " ", regex=True)
mask_non_research = df_filtered_cleaned["PaperTitle"].str.contains(non_research_pattern, case=False, na=False)
non_research_counts = mask_non_research.sum()
df_filtered_cleaned = df_filtered_cleaned[~mask_non_research]
progress_bar.update(1)
print(f"Identified non-research articles: {non_research_counts:,}")

progress_bar.close()

# ========== Compute final counts ==========
# Calculate total removed
total_removed = (
    nonsense_title_counts
    + title_length_counts
    + abstract_length_counts
    + withdraw_counts
    + non_research_counts
)

final_article_count = len(df_filtered_cleaned)

# ========== Final summary & consistency check ==========
removal_log = [
    f"Removed numeric/nonsense titles & placeholders: {nonsense_title_counts:,}",
    f"Removed due to title length issues: {title_length_counts:,}",
    f"Removed due to abstract length issues: {abstract_length_counts:,}",
    f"Removed withdrawn articles: {withdraw_counts:,}",
    f"Removed non-research articles: {non_research_counts:,}",
    f"Final number of articles: {final_article_count:,}",
    f"\nTotal removed (should match sum above): {total_removed:,}"
]

print("\n".join(removal_log))

# Save final cleaned dataset
df_filtered_cleaned.to_csv(os.path.join(output_dir, "searchresults-cleaned.csv"), index=False)

Additional Cleaning:  17%|████████████▎                                                             | 1/6 [00:00<00:00,  5.88step/s]

Identified numeric/nonsense titles: 789


Additional Cleaning:  33%|████████████████████████▋                                                 | 2/6 [00:01<00:02,  1.38step/s]

Identified placeholders: 0


Additional Cleaning:  50%|█████████████████████████████████████                                     | 3/6 [00:02<00:02,  1.35step/s]

...

Invalid Abstract count (too short/long): 10,047


Additional Cleaning:  83%|█████████████████████████████████████████████████████████████▋            | 5/6 [00:16<00:03,  3.92s/step]

Identified withdrawn: 59


Additional Cleaning: 100%|██████████████████████████████████████████████████████████████████████████| 6/6 [00:36<00:00,  6.05s/step]


Identified non-research articles: 2,669
Removed numeric/nonsense titles & placeholders: 789
Removed due to title length issues: 1,734
Removed due to abstract length issues: 10,047
Removed withdrawn articles: 59
Removed non-research articles: 2,669
Final number of articles: 160,045

Total removed (should match sum above): 15,298


In [5]:
output_dir = './output/'
file_path = './output/searchresults-cleaned.csv'
df_filtered_cleaned = pd.read_csv(file_path, lineterminator='\n')

In [6]:
tqdm.pandas()
# Store initial row count before cleaning
cleaning_step2_count = len(df_filtered_cleaned)

# Define cleaning function for "PaperTitle"
def clean_paper_title(title):
    if not isinstance(title, str) or title.strip() == "":
        return None  # Return None for empty or non-string values.

    # Use clean-text to clean the title
    title = clean(
        title,
        fix_unicode=True,  # Fix broken unicode characters
        to_ascii=True,     # Transliterate to closest ASCII characters
        lower=False,       # Preserve original casing
        no_line_breaks=True,  # Remove line breaks
        no_urls=True,      # Remove URLs
        no_emails=True,    # Remove email addresses
        no_phone_numbers=True,  # Remove phone numbers
        no_numbers=False,  # Retain numbers
        no_digits=False,   # Retain digits
        no_currency_symbols=True,  # Remove currency symbols
        no_punct=False
    )

    # Additional custom cleaning specific to titles
    title = title.strip()

    # Remove titles that are just numbers or codes
    if re.fullmatch(r"^\d+$", title):  # Only numbers → Remove
        return None

    # Convert ALL CAPS to Title Case
    if title.isupper():
        title = title.title()

    # Remove non-informative titles
    if len(title) <= 3 or re.fullmatch(r"[^A-Za-z0-9]+", title):
        return None

    return title

# Define cleaning function for "Abstract"
def clean_abstract(abstract):
    if not isinstance(abstract, str) or abstract.strip() == "":
        return None

    abstract = clean(
        abstract,
        fix_unicode=True,
        to_ascii=True,
        lower=False,
        no_line_breaks=False,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="<URL>",
    )

    # Trim whitespace
    abstract = abstract.strip()

    # Remove abstracts that start with specific unwanted text
    blocked_phrases = [
        "Our website uses cookies to enhance your experience.",
        "(Cell Reports"
    ]
    
    # Check if abstract starts with any blocked phrase
    if any(abstract.startswith(phrase) for phrase in blocked_phrases):
        return None  # Mark it as None (to be dropped later)

    return abstract

# Apply cleaning functions

df_filtered_cleaned["PaperTitle"] = df_filtered_cleaned["PaperTitle"].progress_apply(clean_paper_title)
df_filtered_cleaned["Abstract"] = df_filtered_cleaned["Abstract"].progress_apply(clean_abstract)

# Remove rows where "PaperTitle" has 2 or fewer words
def has_min_words(title, min_words=3):
    """Check if title has at least the required number of words."""
    if not isinstance(title, str):
        return False
    return len(title.split()) >= min_words

mask_valid_title = df_filtered_cleaned["PaperTitle"].progress_apply(lambda title: has_min_words(title, 3))
df_filtered_cleaned = df_filtered_cleaned[mask_valid_title]

# Drop any rows where "PaperTitle" or "Abstract" is now empty
df_filtered_cleaned = df_filtered_cleaned.dropna(subset=["PaperTitle", "Abstract"]).reset_index(drop=True)

# Store final row count after cleaning
final_cleaning_step3_count = len(df_filtered_cleaned)
cleaning_step3_rows_removed = cleaning_step2_count - final_cleaning_step3_count

# Save the cleaned dataset
df_filtered_cleaned.to_csv(os.path.join(output_dir,"searchresults-cleaned.csv"), index=False)

# Print summary
print("\nCleaning complete!")
print(f"Initial number of articles: {cleaning_step2_count}")
print(f"Final number of articles after cleaning: {final_cleaning_step3_count}")
print(f"Total number of removed articles: {cleaning_step3_rows_removed}")

100%|███████████████████████████████████████████████████████████████████████████████████| 160045/160045 [00:00<00:00, 779323.58it/s]



Cleaning complete!
Initial number of articles: 160045
Final number of articles after cleaning: 159950
Total number of removed articles: 95


In [7]:
# More specific normalization of PaperTitle and Abstracts, without removing any furhter columns
    
# Store initial row count before cleaning
cleaning_step3_count = len(df_filtered_cleaned)

# Enable tqdm progress bars for pandas operations
tqdm.pandas()

# Helper function to convert ALL CAPS text to sentence case
def to_sentence_case(text):
    if text.isupper():  # Check if it's all caps
        return text.capitalize()  # Convert to sentence case
    return text  # Return unchanged if not all caps


# Cleaning function for PaperTitle
def clean_paper_title(title):
    if not isinstance(title, str) or title.strip() == "":
        return title 
    
    # Remove unwanted keywords at the beginning of the title
    title = re.sub(
        r"^\s*(\*?\[?(invited|keynote|winner|regular paper|blog)\]?\*?)\s*[:\-]*",
        "",
        title,
        flags=re.IGNORECASE
    )
    title = re.sub(r"[_-]{5,}", "", title)  # Remove long underscores or dashes
    title = re.sub(r"^\s*[_]*Abstract[_]*\s*[:\-]*", "", title, flags=re.IGNORECASE)  # Remove "Abstract" heading
    title = re.sub(r"^\s*[\.\,\-\:\;\//\=\s]+", "", title)  # Remove leading . , - : ; // = and spaces
    title = re.sub(r"^\[([^\]]+)\]\.?\s*$", r"\1", title)  # Remove surrounding square brackets + trailing dot
    title = re.sub(r"\s+", " ", title).strip()  # Normalize spaces

    title = to_sentence_case(title)  # Convert to sentence case if all caps

    return title

# Cleaning function for Abstract
def clean_abstract(abstract):
    if not isinstance(abstract, str) or abstract.strip() == "":
        return abstract

    # Remove everything before "introduction" if abstract starts with "You have access"
    if abstract.lower().startswith("you have access"):
        match = re.search(r"\bintroduction\b", abstract, re.IGNORECASE)
        if match:
            abstract = abstract[match.start():]  # Keep everything from "introduction" onwards

    # If it starts with "//", remove everything until "abstract" appears
    if abstract.startswith("//"):
        match = re.search(r"\babstracts?\b", abstract, re.IGNORECASE)
        if match:
            abstract = abstract[match.start():]  # Keep only the part after "abstract"

    # Remove leading colons and spaces
    abstract = re.sub(r"^\s*[:]+\s*", "", abstract)  # Remove any leading : and spaces

    # Remove long underscores/dashes at the start
    abstract = re.sub(r"[_-]{5,}", "", abstract)  # Remove long underscores or dashes
    abstract = re.sub(r"^\s*[_]*Abstract[_]*\s*[:\-]*", "", abstract, flags=re.IGNORECASE)  # Remove "Abstract" heading
    abstract = re.sub(r"^\s*[\.\,\-\:\;\=\s]+", "", abstract)  # Remove leading . , - : ; = and spaces
    abstract = re.sub(r"\s*<[^>]+>\s*", " ", abstract)  # Remove XML-like tags like <CUR>
    abstract = re.sub(r"[,\.]{2,}", ".", abstract)  # Normalize excessive dots and commas
    abstract = re.sub(r"\s+", " ", abstract).strip()  # Normalize spaces

    abstract = to_sentence_case(abstract)  # Convert to sentence case if all caps

    return abstract

# Apply cleaning with progress bar
print("\nCleaning 'PaperTitle' column...")
df_filtered_cleaned["PaperTitle"] = df_filtered_cleaned["PaperTitle"].progress_apply(clean_paper_title)

print("\nCleaning 'Abstract' column...")
df_filtered_cleaned["Abstract"] = df_filtered_cleaned["Abstract"].progress_apply(clean_abstract)

# Store initial row count after cleaning
cleaning_step4_count = len(df_filtered_cleaned)
cleaning_step4_rows_removed = cleaning_step3_count - cleaning_step4_count

# Save cleaned version
df_filtered_cleaned.to_csv('./output/searchresults-cleaned.csv', index=False)


# Print summary
print("\nNormalization complete!")
print(f"Count before cleaning step 4: {cleaning_step3_count:,}")
print(f"Count after cleaning step 4: {cleaning_step4_count:,}")



Cleaning 'PaperTitle' column...


100%|████████████████████████████████████████████████████████████████████████████████████| 159950/159950 [00:01<00:00, 91226.91it/s]



Cleaning 'Abstract' column...


100%|█████████████████████████████████████████████████████████████████████████████████████| 159950/159950 [00:29<00:00, 5335.87it/s]



Normalization complete!
Count before cleaning step 4: 159,950
Count after cleaning step 4: 159,950
